<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#testdata" data-toc-modified-id="testdata-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>testdata</a></span></li><li><span><a href="#real-data" data-toc-modified-id="real-data-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>real data</a></span></li></ul></li><li><span><a href="#question-1" data-toc-modified-id="question-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>question 1</a></span></li><li><span><a href="#question-2" data-toc-modified-id="question-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>question 2</a></span></li></ul></div>

In [ ]:
import numpy as np
from sympy.utilities.iterables import multiset_permutations
from functools import lru_cache

### testdata

In [ ]:
testdata = """???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""".split("\n")
testdata = [ dat.split() for dat in testdata ]
testdata

In [ ]:
data = [ [np.array(list(dat[0])), list(map(int, dat[1].split(',')))] for dat in testdata]
springs, Ns = zip(*data)

### real data

In [ ]:
with open("input.txt") as f:
    data = [dat.strip().split() for dat in f ]
data = [ [np.array(list(dat[0])), list(map(int, dat[1].split(',')))] for dat in data]
springs, Ns = zip(*data)

In [ ]:
len(springs)

## question 1

In [ ]:
def get_options(springs, N):
    Ntot = sum(N)
    whereQuest = np.where(springs=='?')
    nQuest, nHash = np.sum(springs=='?'), np.sum(springs=='#')
    #print(springs, nQuest, nHash)
    nNewHash = Ntot - nHash
    nNewDots = nQuest - nNewHash
    newSymbols = nNewHash * "#" + nNewDots * "."
    
    nValidOptions = 0
    for option in multiset_permutations(newSymbols):
        # fill them in.
        new_springs = springs.copy()
        new_springs[whereQuest] = option
        #print(option, new_springs)
        
        # validate
        newN = np.array(list(map(len, "".join(new_springs).split("."))))
        newN = newN[newN!=0]
        #print(newN)
        if len(newN)==len(N):
            if np.array_equal(newN, N):
                nValidOptions += 1
    return nValidOptions

In [ ]:
Nall = 0
for spring, N in zip(springs, Ns):
    #print("".join(spring), N)
    n = get_options(spring, N)
    print(n)
    Nall += n
Nall

## question 2

In [ ]:
@lru_cache
def count(cfg, nums):
    if cfg == "":
        return 1 if nums == () else 0
    if nums == ():
        return 0 if "#" in cfg else 1
    
    key = (cfg, nums)
    
    result = 0
    
    # add the number of possibilities starting with a dot, or with regarding the ? as a dot.
    if cfg[0] in ".?":
        result += count(cfg[1:], nums)
    # add the number of possibilities starting with a hashtag or with regarding ? as #    
    if cfg[0] in "#?":
        # stop when:
        # - the first number is larger than the number of symbols left.
        # - there is a dot within the range of the next number
        # - cfg[nums[0]] != "#"
        if nums[0] <= len(cfg) and "." not in cfg[:nums[0]] and (nums[0] == len(cfg) or cfg[nums[0]] != "#"):
        #if nums[0] <= len(cfg) and "." not in cfg[:nums[0]]:
            result += count(cfg[nums[0] + 1:], nums[1:])
    return result

In [ ]:
Nall = 0
for spring, N in zip(springs, Ns):
    nums = tuple(N * 5)
    cfg = "?".join("".join(spring) for _ in range(5))
    n = count(cfg, nums)
    #print(n)
    Nall += n
Nall

In [ ]:
10153896718999